In [1]:
!git clone https://github.com/m-xrtin/TikTok-TechJam-2025.git
%cd TikTok-TechJam-2025

Cloning into 'TikTok-TechJam-2025'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 92 (delta 37), reused 68 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 1.53 MiB | 10.49 MiB/s, done.
Resolving deltas: 100% (37/37), done.
/content/TikTok-TechJam-2025


In [5]:
%cd data
!ls

/content/TikTok-TechJam-2025/data
review-Alaska_meta.json		      review-Kaggle_meta.json
review-Alaska_standardized.json       review-New_York_meta1.json
review-California_meta1.json	      review-New_York_meta2.json
review-California_meta2.json	      review-New_York_standardized1.json
review-California_standardized1.json  review-New_York_standardized2.json
review-California_standardized2.json


In [47]:
import pandas as pd

df = pd.read_json("review-Alaska_meta.json", lines=False)  # try lines=True if error
print(df.head())
print(df.columns)

        user_id        user_name  \
0  1.155657e+20     Ana Viaviase   
1  1.034519e+20    Carter Finney   
2  1.139508e+20  Jeremy Reynolds   
3           NaN            AmitB   
4  1.132674e+20    Carissa Toner   

                                       business_name           time  \
0                    Olive Garden Italian Restaurant  1553466571237   
1                        Ammo-Can Coffee Social Club  1560922731421   
2                                            Verizon  1493661397977   
3            Courtyard by Marriott Anchorage Airport  1439856000000   
4  Diamond Janitorial Services - Pre-Listing Clea...  1616205749964   

                                                text  rating  \
0                                      Service sucks     1.0   
1                                               None     5.0   
2  It's a cellular store it's clean an the people...     5.0   
3  Excellent location. Not only close to the airp...     NaN   
4  She cancelled my appointment next

In [48]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import pipeline
from pylab import rcParams

In [55]:
# 1) Define labels and grab one example text safely
LABELS = ["genuine review", "fake or manipulated review"]
i = 0  # pick any index you like

text = df.loc[i, "text"] if ("text" in df.columns and i in df.index) else ""
text = "" if text is None else str(text).strip()
if not text:
    raise ValueError(f"Row {i} has empty/None text. Pick another index.")

# 2) Build zero-shot pipeline on GPU if available (use device=0), else CPU with -1
clf = pipeline(
    "zero-shot-classification",
    model="typeform/distilbert-base-uncased-mnli",  # fast; switch to bart-large-mnli for stronger
    device=0  # set to -1 for CPU
)

# 3) Call the pipeline in a version-safe way
try:
    # Newer HF uses candidate_labels=
    out = clf(text, candidate_labels=LABELS, multi_label=False, hypothesis_template="This text is {}.")
except TypeError:
    # Some versions expect labels=
    out = clf(text, labels=LABELS, hypothesis_template="This text is {}.")

print(out)  # full dict
print("Top label:", out["labels"][0], "| score:", float(out["scores"][0]))

# Optional: get just the fake probability
fake_label = "fake or manipulated review"
fake_idx = out["labels"].index(fake_label)
fake_prob = float(out["scores"][fake_idx])
print("fake_prob:", fake_prob)

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


{'sequence': 'Service sucks', 'labels': ['fake or manipulated review', 'genuine review'], 'scores': [0.8264290690422058, 0.17357094585895538]}
Top label: fake or manipulated review | score: 0.8264290690422058
fake_prob: 0.8264290690422058


In [10]:
#Combine all provided datasets to perform BERT transformer analysis
import glob

files = glob.glob("*.json")

dfs = []
for f in files:
  #check the formatting of json files, for debugging
    try:
        temp = pd.read_json(f, lines=False)
        temp["source_file"] = f
        dfs.append(temp)
    except Exception as e:
        print("Error reading", f, e)

df_all = pd.concat(dfs, ignore_index=True)
print("Combined shape:", df_all.shape)
print(df_all.head())
print(df_all.columns)

Combined shape: (11000, 10)
        user_id       user_name business_name          time  \
0  1.046116e+20    Juana Patton           NaN  1.507068e+12   
1  1.181364e+20    Matthew Soby           NaN  1.555012e+12   
2  1.009220e+20     Len Woodrow           NaN  1.614283e+12   
3  1.011883e+20       i-fan Hsu           NaN  1.615097e+12   
4  1.178791e+20  Randell Haynes           NaN  1.536937e+12   

                                                text  rating  \
0                                               None     1.0   
1  Unique fushion of Indian spice and CALI style....     5.0   
2  Great bread, spread, appetizers. The salads ar...     3.0   
3                                               None     4.0   
4                                               None     4.0   

  sentiment_category rating_category                                gmap_id  \
0                NaN        negative  0x80c2c8313830e76d:0x2f8f6c717afd95d3   
1                NaN        positive  0x808fbf1d61

In [43]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 2.5 MB/s eta 0:00:00


In [46]:
import numpy as np
from transformers import pipeline

sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

# 1) Build a mask of valid, non-empty texts
if "text" not in df_all.columns:
    df_all["text"] = np.nan

# Make sure it's a string column (without turning NaN into the literal "nan")
text_str = df_all["text"].astype("string")
mask = text_str.notna() & (text_str.str.strip() != "")

# 2) Run the model in batches (much faster than row-wise .apply)
valid_texts = text_str[mask].tolist()
if valid_texts:
    outputs = sentiment_analyzer(valid_texts, truncation=True)
    df_all.loc[mask, "roberta_sentiment"] = [o[0]["label"] for o in outputs]
    df_all.loc[mask, "roberta_score"]     = [o[0]["score"] for o in outputs]

# 3) Fill placeholders for empty/None texts
df_all.loc[~mask, "roberta_sentiment"] = "EMPTY"
df_all.loc[~mask, "roberta_score"] = np.nan

# 4) Show just the first few rows
print(df_all[["text", "roberta_sentiment", "roberta_score"]].head(10))


Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


RuntimeError: The expanded size of the tensor (1144) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1144].  Tensor sizes: [1, 514]

In [40]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax


#implement RoBERTa transformer to fill out sentiment for dataset
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
tok = AutoTokenizer.from_pretrained(MODEL)
modl = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [20]:
example = df_all['text'][1]
print(example)

Unique fushion of Indian spice and CALI style. The naan taco was perfect for lunch. Nice to have a quick alternative.


In [27]:
#Run RoBERTa model
encoded_text = tokenizer(example, return_tensors='pt')
output = model(**encoded_text)
scores = output[0][0].detach().numpy()
scores = softmax(scores)
#makes our scores interpretable as probabilities
scores_dict = {
    'RB_neg' : scores[0],
    'RB_neu' : scores[1],
    'RB_pos' : scores[2]
}

array([0.001303  , 0.01931093, 0.97938603], dtype=float32)

In [30]:
def polarity_scores_roberta(example):
  encoded_text = tokenizer(example, return_tensors='pt')
  output = model(**encoded_text)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  #makes our scores interpretable as probabilities
  scores_dict = {
      'RB_neg' : scores[0],
      'RB_neu' : scores[1],
      'RB_pos' : scores[2]
  }
  return scores_dict

In [33]:
!pip install tqdm
from tqdm import tqdm

In [38]:
res = {}

for i, row in tqdm(df.iterrows(), total = len(df_all)):
  text = row['text']
  roberta_result = polarity_scores_roberta(text)
  res[i] = {**roberta_result}

  0%|          | 1/11000 [00:00<19:13,  9.54it/s]


ValueError: You need to specify either `text` or `text_target`.

In [35]:
roberta_result

{'RB_neg': np.float32(0.9643311),
 'RB_neu': np.float32(0.030233998),
 'RB_pos': np.float32(0.0054349434)}

In [41]:
label_map = {0: "negative", 1: "neutral", 2: "positive"}

def roberta_sentiment_batch(texts, batch_size=64, max_length=256):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    out_rows = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            chunk = texts[i:i+batch_size]
            enc = tok(chunk, return_tensors="pt", truncation=True, padding=True, max_length=max_length).to(device)
            logits = mdl(**enc).logits
            probs = torch.softmax(logits, dim=-1).cpu().numpy()
            for p in probs:
                neg, neu, pos = p.tolist()
                comp = float(pos - neg)  # simple monotonic "compound"
                pred_idx = int(p.argmax())
                out_rows.append({
                    "rb_neg": neg,
                    "rb_neu": neu,
                    "rb_pos": pos,
                    "rb_compound": comp,
                    "rb_label": label_map[pred_idx]
                })
    return pd.DataFrame(out_rows)

# Ensure text is string
texts = df_all["text"].astype(str).tolist()
roberta_df = roberta_sentiment_batch(texts, batch_size=64)

# Attach to your frame
df_all = pd.concat([df_all.reset_index(drop=True), roberta_df.reset_index(drop=True)], axis=1)


KeyboardInterrupt: 